In [1]:
import random
from pathlib import Path

import cv2
import numpy as np

In [ ]:
def motion_kernel(length: int = 15, angle: float = 0.0) -> np.ndarray:
    """
    Create a normalized 2D motion blur kernel of given length and angle (degrees).
    """
    length = max(3, int(length))
    if length % 2 == 0:
        length += 1  # make it odd

    kernel = np.zeros((length, length), dtype=np.float32)

    cv2.line(kernel, (0, length // 2), (length - 1, length // 2), 1.0, thickness=1)

    center = (length / 2 - 0.5, length / 2 - 0.5)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (length, length))

    kernel_sum = kernel.sum()
    if kernel_sum != 0:
        kernel /= kernel_sum

    return kernel

def apply_random_motion_blur(
    img_bgr: np.ndarray,
    min_length: int = 5,
    max_length: int = 35
) -> tuple[np.ndarray, int, float]:
    """
    Apply random motion blur to an image.

    Returns:
        blurred_img, length, angle
    """
    length = random.randint(min_length, max_length)
    angle = random.uniform(0, 180)  # degrees

    k = motion_kernel(length, angle)
    blurred = cv2.filter2D(img_bgr, -1, k)

    return blurred, length, angle

In [ ]:
input_dir = Path(r".\raw\images")
output_dir = Path(r".\processed_300")

output_dir.mkdir(parents=True, exist_ok=True)

image_paths = (
    list(input_dir.glob("*.jpg")) +
    list(input_dir.glob("*.jpeg")) +
    list(input_dir.glob("*.JPG")) +
    list(input_dir.glob("*.JPEG"))
)

print(f"Found {len(image_paths)} images in {input_dir}")

if not image_paths:
    raise ValueError("No JPEG images found – check your input_dir path.")

random.seed(42)

print("Applying random motion blur to ALL images in the directory...")

for img_path in image_paths:
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Warning: could not read {img_path}, skipping.")
        continue

    blurred, length, angle = apply_random_motion_blur(img)

    out_name = f"{img_path.stem}.jpg"
    out_path = output_dir / out_name

    success = cv2.imwrite(str(out_path), blurred)
    if success:
        print(f"Saved: {out_path}")
    else:
        print(f"Failed to save: {out_path}")

Found 600 images in raw\images
Applying random motion blur to ALL images in the directory...
Saved: processed_300\-_jpg.rf.6080835926ae88d6b66f5e5619f9549e.jpg
Saved: processed_300\000f78a66fd29c49_jpg.rf.c6cbedf1a9663d16a62fe841efb514d2.jpg
Saved: processed_300\000f9ebbd5fc3745_jpg.rf.83d368395431b16bd87650799f832a1b.jpg
Saved: processed_300\0033d09deafc7bcd_jpg.rf.b451aea4d1a1edfc994cd2909d73ab3e.jpg
Saved: processed_300\003b80fc0494a9f4_jpg.rf.1efe60adee80950c5bf0eb800b78002f.jpg
Saved: processed_300\004e556570882f12_jpg.rf.bd8aff2e3ca35ee40c6c0ce373b3cd9d.jpg
Saved: processed_300\005cb9e795fa91de_jpg.rf.b2b84fdedd7a2477b16805395c72f260.jpg
Saved: processed_300\007b3d748a2d2396_jpg.rf.acf6a55895bae1a6f4f78fcb97057039.jpg
Saved: processed_300\00c68c590d7e3ec0_jpg.rf.e3f1d110d28834e0eeade0ba460a3090.jpg
Saved: processed_300\00d168fbb8a96a7d_jpg.rf.29d7079ee9a374a05efcdf43fca84911.jpg
Saved: processed_300\00d2c29446861097_jpg.rf.2c881a566f9adcf7f9f3aeae8725a48b.jpg
Saved: processed_300